In [18]:
import os
from decouple import config
from langchain_community.vectorstores import UpstashVectorStore
from langchain_community.embeddings.yandex import YandexGPTEmbeddings
from langchain_community.llms import YandexGPT
from langchain_core.prompts import ChatPromptTemplate

In [3]:
UPSTASH_VECTOR_REST_URL = config("UPSTASH_VECTOR_REST_URL")
UPSTASH_VECTOR_REST_TOKEN = config("UPSTASH_VECTOR_REST_TOKEN")
STORE_CONFIG = {
    "index_url": UPSTASH_VECTOR_REST_URL,
    "index_token": UPSTASH_VECTOR_REST_TOKEN
}

In [4]:
YC_FOLDER_ID = config('YC_FOLDER_ID')
YC_API_KEY = config('YC_API_KEY')
YA_EMB_CONFIG = {
    "api_key": YC_API_KEY,
    "folder_id": YC_FOLDER_ID,
    "doc_model_name": "text-search-query"
}

In [15]:
YC_FOLDER_ID = config('YC_FOLDER_ID')
YC_API_KEY = config('YC_API_KEY')
LLM_CONFIG = {
    "api_key": YC_API_KEY,
    "folder_id": YC_FOLDER_ID,
    "model_name": "yandexgpt-lite"
}

In [17]:
embeddings = YandexGPTEmbeddings(**YA_EMB_CONFIG)
store = UpstashVectorStore(embedding=embeddings, **STORE_CONFIG)
llm = YandexGPT(**LLM_CONFIG)

In [7]:
retriever = store.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 2}
)

In [11]:
result = retriever.invoke("Бургер?")

In [19]:
message = """
Ответь на вопрос, используя только предоставленный контекст.

Вопрос: {question}

Контекст: {context}
"""

In [20]:
prompt = ChatPromptTemplate.from_messages([('human', message)])

In [21]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [22]:
runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: x["num"] * 123121
)

In [23]:
runnable.invoke({'num': 31})

{'passed': {'num': 31}, 'modified': 3816751}

In [24]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [26]:
chain = {'context': retriever, "question": RunnablePassthrough()} | prompt | llm | parser

In [42]:
chain.invoke('Какое население в Питере?')

'В источнике нет информации о населении Санкт-Петербурга.\n\nЕсли у вас есть другой вопрос, я постараюсь на него ответить.'

In [41]:
llm.invoke('Какое население в Питере?')

'По данным Росстата, население Санкт-Петербурга на 1 января 2023 года составляло около 5 468 382 человек.'